# Compute Shap of RF

In [1]:
%pwd

'/home/rapids/notebooks/DP15/03_RStudio/PyCode_v241111'

In [2]:
%cd ..

/home/rapids/notebooks/DP15/03_RStudio


## Import Package

In [3]:
import cudf
from cuml.ensemble import RandomForestRegressor
from cuml.explainer import KernelExplainer
from glob import glob
from joblib import dump, load
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import KFold

In [4]:
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

## Load Data

In [5]:
### X and y
def getXandY(Output_Vari):
    y_list = glob("01_Data/*_y_" + Output_Vari + "*.csv")
    y = pd.read_csv(y_list[0], index_col=0)
    y = y.iloc[:,0].to_numpy()
    X_list = glob("01_Data/*_X_" + Output_Vari + "*.csv")
    X = pd.read_csv(X_list[0], index_col=0)
    return X, y

In [6]:
Output_Vari = "Happinessoverall"

In [7]:
X, y = getXandY(Output_Vari)

## Shap

In [8]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [9]:
X.shape

(383173, 29)

In [ ]:
Shap_df = []

i = 1
for train_index, test_index in kf.split(X):
    print(f"This is {i} fold")
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y[train_index], y[test_index]

    cX_train = cudf.from_pandas(X_train)
    cX_test = cudf.from_pandas(X_test)
    cy_train = cudf.Series(y_train)
    cy_test = cudf.Series(y_test)
    
    # Train the model
    model = RandomForestRegressor(n_estimators = 3800, min_samples_split = 6, max_features = 0.5, max_depth = 14, split_criterion=2)
    model.fit(cX_train, cy_train)

    colnames = X_test.columns + '_shap'

    background = cudf.from_pandas(X_train.iloc[:20,:])
    cu_explainer = KernelExplainer(model=model.predict,
                                   data=background,
                                   is_gpu_model=True, random_state=42,
                                   nsamples = 1_000)
    
    interval = 1000
    for row in list(range(0, X_test.shape[0], interval)):
        print(f'here{row}:{row + interval}')
        %time cu_shap_value_merge = cu_explainer.shap_values(X_test.iloc[row:row + interval,:])
        X_test_shap = pd.DataFrame(cu_shap_value_merge, columns=colnames, index=X_test.index[row:row + interval])
        X_test_merge = pd.concat([X_test.iloc[row:row + interval,:], X_test_shap], axis=1)
        X_test_merge['y'] = y_test[row:row + interval]

        X_test_merge.to_parquet(f'03_Results/ShapAll/X_test_Fold_{i}_{row}_{row + interval}.parquet')
        Shap_df.append(X_test_merge)

    dump(Shap_df, f"03_Results/Shap_fold_{i}_Dataset.joblib")

    i = i + 1

This is 1 fold
here0:1000
